# Storm Surge Alerts Feeds

## Development Notes

Notes about development of ATOM/RSS feeds for storm surge alerts.
Started in the process of working out the details of how to provide
a feed to Port Metro Vancouver during the 2015/2016 storm surge season.
Consideration is also given to the possibility of providing a wider collection
of feeds in the future.

Port Metro Vancouver presently consumes [Environment Canada feeds](https://weather.gc.ca/business/index_e.html),
so we use that as our pattern.

Feeds are generically referred to as RSS,
but there are 2 current standards for them:
[ATOM](https://en.wikipedia.org/wiki/Atom_%28standard%29)
and [RSS-2.0](https://en.wikipedia.org/wiki/RSS).
EC uses ATOM,
and it seems to be somewhat more favourable technically,
so that's what we'll focus on initially.

The IETF standard for ATOM is [RFC 4287](https://tools.ietf.org/html/rfc4287).

The [Vancouver weather forecast web page](http://weather.gc.ca/city/pages/bc-74_metric_e.html) 
has links to the [weather forecast feed](http://weather.gc.ca/rss/city/bc-74_e.xml),
and the [weather alerts feed](http://weather.gc.ca/rss/warning/bc-74_e.xml).
Using the Firefox "View Page Source" function on the page that loads from either of those
feed links lets you see the structure of the feed content.

Other useful links:

* A blog post about [creating ATOM ID elements](http://web.archive.org/web/20110514113830/http://diveintomark.org/archives/2004/05/28/howto-atom-id)

* A blog post (linked from the above one) about [using tag URIs as ATOM IDs](http://web.archive.org/web/20110514113830/http://www.taguri.org/)

* The Python `feedgen` package [documentation](http://lkiesow.github.io/python-feedgen/),
[Github repository](https://github.com/lkiesow/python-feedgen),
and [PyPI page](https://pypi.python.org/pypi/feedgen/).

### URLs

The storm surge alerts feeds will be located under http://salishsea.eos.ubc.ca/storm-surge/.

The initial ATOM feed for Port Metro Vancouver (PMV) will be http://salishsea.eos.ubc.ca/storm-surge/atom/pmv.xml.

That pattern can be extended to provide other feed like:
* One containing alert messages for the entire model domain
based on Ben's narrative template developed during the Aug-2015 sprint:
http://salishsea.eos.ubc.ca/storm-surge/atom/alerts.xml
* Feeds for specific locations: http://salishsea.eos.ubc.ca/storm-surge/atom/PointAtkinson.xml
* Feeds customized for other stakeholders

This URL structure also allows for the possibility of generating RSS-2.0 feeds in the future
at locations like:
* http://salishsea.eos.ubc.ca/storm-surge/rss/pmv.xml
* http://salishsea.eos.ubc.ca/storm-surge/rss/alerts.xml
* http://salishsea.eos.ubc.ca/storm-surge/rss/PointAtkinson.xml
* etc.

Even though the PMV and Point Atkinson feeds are for the same geolocation 
and are expressions of the same model results,
they are treated separately so that the PMV feed can be customized with
units,
threshold levels,
etc.
requested by that stakeholder.

### Generating the PMV Feed

EC appears to generate a new feed each time the [forecast page](http://weather.gc.ca/city/pages/bc-74_metric_e.html)
is updated
(so, hourly, at least)
rather than appeding new feed entries to an existing feed.
We will adopt the same practice for the PMV feed:
* A new feed will be generated each day after the forecast and forecast2 runs
* In contrast to EC, who always have at least new current conditions to report on,
we will only generate feed entries when the Point Atkinson water level is forecast to
exceed the risk and extreme risk thresholds

We'll use the [`feedgen` package](http://lkiesow.github.io/python-feedgen/)
to construct the feed and its entries and store them to disk.

In [1]:
from pprint import pprint

import arrow
from feedgen.feed import FeedGenerator

In [20]:
fg = FeedGenerator()

utcnow = arrow.utcnow()

fg.title('Salish Sea NEMO Model Storm Surge Alerts for Port Metro Vancouver')
fg.id(
    'tag:salishsea.eos.ubc.ca,2015-12-12:/storm-surge/atom/pmv/{utcnow}'
    .format(utcnow=utcnow.format('YYYYMMDDHHmmss')))
fg.language('en-ca')
fg.author(name='Salish Sea MEOPAR Project', uri='http://salishsea.eos.ubc.ca/')
fg.rights(
    'Copyright {this_year}, Salish Sea MEOPAR Project Contributors and The University of British Columbia'
    .format(this_year=utcnow.year))
fg.link(href='http://salishsea.eos.ubc.ca/storm-surge/atom/pmv.xml', rel='self', type='application/atom+xml')
fg.link(href='http://salishsea.eos.ubc.ca/storm-surge/forecast.html', rel='related', type='text/html')

pprint(fg.atom_str(pretty=True).decode('ascii'))

("<?xml version='1.0' encoding='UTF-8'?>\n"
 '<feed xmlns="http://www.w3.org/2005/Atom" xml:lang="en-ca">\n'
 '  '
 '<id>tag:salishsea.eos.ubc.ca,2015-12-12:/storm-surge/atom/pmv/20151213192409</id>\n'
 '  <title>Salish Sea NEMO Model Storm Surge Alerts for Port Metro '
 'Vancouver</title>\n'
 '  <updated>2015-12-13T19:24:09.062955+00:00</updated>\n'
 '  <author>\n'
 '    <name>Salish Sea MEOPAR Project</name>\n'
 '    <url>http://salishsea.eos.ubc.ca/</url>\n'
 '  </author>\n'
 '  <link href="http://salishsea.eos.ubc.ca/storm-surge/atom/pmv.xml" '
 'rel="self" type="application/atom+xml"/>\n'
 '  <link href="http://salishsea.eos.ubc.ca/storm-surge/forecast.html" '
 'rel="related" type="text/html"/>\n'
 '  <generator version="0.3.2">python-feedgen</generator>\n'
 '  <rights>Copyright 2015, Salish Sea MEOPAR Project Contributors and The '
 'University of British Columbia</rights>\n'
 '</feed>\n')


Instead of pretty-printing the ASCII version of the feed,
the production code will save it as binary data in a file with:
```
fg.atom_file(os.path.join(path_to_storm_surge, 'atom', 'pmv.xml')
```

The only really interesting bit in the code above is the calculation of the `id` element for the feed.
Quoting [Mark Pilgrim's blog post](http://web.archive.org/web/20110514113830/http://diveintomark.org/archives/2004/05/28/howto-atom-id):

> There are three requirements for an Atom ID:
>
>    1. The ID must be a valid URI, as defined by RFC 2396.
>    2. The ID must be globally unique, across all Atom feeds, everywhere, for all time. This part is actually easier than it sounds.
>    3. The ID must never, ever change.

We use the [tag URI](http://web.archive.org/web/20110514113830/http://www.taguri.org/)
technique to create our `id` with the EC tactic of appending the UTC feed create date/time
as a string of digits:

`tag:salishsea.eos.ubc.ca,2015-12-12:/storm-surge/atom/pmv/20151213182007`

This tag is composed of:
* Our domain name: `salishsea.eos.ubc.ca`
* A comma
* The date on which this scheme was conceived: `2015-12-12`
* A colon
* The path to the feed file, excluding the `.xml` extension: `/storm-surge/atom/pmv`
* The UTC date/time when the feed was created as a string of digits, prepended with a '/': `/20151213182007`

### Generating a PMV Feed Entry

The feed above will be generated after every forecast and forecast2 run
by the `make_feeds` worker between the successful completion of the 
`make_site_page forecast publish` worker and the launch of the `push_to_web` worker.

If the Point Atkinson water level is forecast to exceed the risk and extreme risk thresholds,
the `make_feeds` worker will also add an entry to the feed.
Apart from the calculation of the alert text for the entry,
its creation looks like:

In [21]:
fe = fg.add_entry()

now = arrow.now()

fe.title('Storm Surge Alert for Point Atkinson')
fe.id(
    'tag:salishsea.eos.ubc.ca,{today}:/storm-surge/atom/pmv/{now}'
    .format(
        today=now.format('YYYY-MM-DD'),
        now=now.format('YYYYMMDDHHmmss')))
fe.author(name='Salish Sea MEOPAR Project', uri='http://salishsea.eos.ubc.ca/')
fe.summary('')
fe.link(href='salishsea.eos.ubc.ca/nemo/results/forecast/publish_14dec15.html', rel='alternate', type='text/html')

pprint(fg.atom_str(pretty=True).decode('ascii'))

("<?xml version='1.0' encoding='UTF-8'?>\n"
 '<feed xmlns="http://www.w3.org/2005/Atom" xml:lang="en-ca">\n'
 '  '
 '<id>tag:salishsea.eos.ubc.ca,2015-12-12:/storm-surge/atom/pmv/20151213192409</id>\n'
 '  <title>Salish Sea NEMO Model Storm Surge Alerts for Port Metro '
 'Vancouver</title>\n'
 '  <updated>2015-12-13T19:24:09.062955+00:00</updated>\n'
 '  <author>\n'
 '    <name>Salish Sea MEOPAR Project</name>\n'
 '    <url>http://salishsea.eos.ubc.ca/</url>\n'
 '  </author>\n'
 '  <link href="http://salishsea.eos.ubc.ca/storm-surge/atom/pmv.xml" '
 'rel="self" type="application/atom+xml"/>\n'
 '  <link href="http://salishsea.eos.ubc.ca/storm-surge/forecast.html" '
 'rel="related" type="text/html"/>\n'
 '  <generator version="0.3.2">python-feedgen</generator>\n'
 '  <rights>Copyright 2015, Salish Sea MEOPAR Project Contributors and The '
 'University of British Columbia</rights>\n'
 '  <entry>\n'
 '    '
 '<id>tag:salishsea.eos.ubc.ca,2015-12-13:/storm-surge/atom/pmv/20151213112410</id

Each entry also requires a unique `id`.
Here we use a similar algorithm to the feed `id` except that the date/time is local rather than UTC:

`tag:salishsea.eos.ubc.ca,2015-12-13:/storm-surge/atom/pmv/20151213110621`

This tag is composed of:
* Our domain name: `salishsea.eos.ubc.ca`
* A comma
* The date on which the entry was created: `2015-12-13`
* A colon
* The path to the feed file, excluding the `.xml` extension: `/storm-surge/atom/pmv`
* The local date/time when the entry was created as a string of digits, prepended with a '/': `/20151213110621`

### Calculation of the Alert Text for the Entry `summary`

TODO